In [0]:
import warnings
warnings.filterwarnings("ignore")
from keras.layers import Conv2D, MaxPooling2D, Dropout, Input, concatenate, UpSampling2D, Activation, Reshape, Conv2DTranspose
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import Sequence
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from google.colab import files, drive
from skimage.io import imread, imsave
from skimage.transform import resize
from matplotlib import pyplot as plt
import numpy as np
import os
import io
import zipfile
import cv2
from matplotlib import pyplot as plt

Using TensorFlow backend.


In [0]:
drive.mount('/content/gdrive', force_remount=True)
root_path = 'gdrive/My Drive/tabOCR/'

Mounted at /content/gdrive


In [0]:
#uploaded = files.upload()
#data = zipfile.ZipFile(io.BytesIO(uploaded['dataset_ann.zip']), 'r')
#data.extractall()

In [0]:
input_height = 1024
input_width = 1024
n_classes = 6
batch_size = 4
loss = 'categorical_crossentropy'

In [0]:
# generator

In [0]:
class image_generator(Sequence):

    def __init__(self, image_filenames, mask_filenames, batch_size, img_size, n_calsses, loss='categorical_crossentropy', no_reshape=False):
        self.image_filenames, self.mask_filenames = image_filenames, mask_filenames
        self.batch_size = batch_size
        self.height = img_size[0]
        self.width = img_size[1]
        self.n_calsses = n_calsses
        self.no_reshape = no_reshape
        self.loss = loss

    def __len__(self):
        return int(np.ceil(len(self.image_filenames) / float(self.batch_size)))

    def get_img_array(self, file_name):
      img = cv2.imread(file_name, 3)
      return cv2.resize(img, (self.height, self.width)).astype('float32')

    def get_segmentation_mask(self, file_name):
        img = cv2.imread(file_name, 1)
        img = cv2.resize(img, (self.height, self.width), interpolation=cv2.INTER_NEAREST)
        img = img[:, :, 0]
        seg_labels = np.zeros((self.width, self.height, self.n_calsses))
        for c in range(self.n_calsses):
          seg_labels[:, :, c] = (img == c).astype(int)
        if not self.no_reshape:
          seg_labels = np.reshape(seg_labels, (self.height*self.width, self.n_calsses))
        return seg_labels

    def __getitem__(self, idx):
        batch_x = self.image_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.mask_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]

        if self.loss == 'categorical_crossentropy':
           return (np.array([self.get_img_array(file_name) for file_name in batch_x]),
                   np.array([self.get_segmentation_mask(file_name) for file_name in batch_y]))
        else:
          return (np.array([self.get_img_array(file_name)for file_name in batch_x]),
                  np.array([self.get_segmentation_mask(file_name) for file_name in batch_y]))

In [0]:
# from drive
'''
image_filenames = []
mask_filenames = []
for path, _, files in os.walk(root_path + "table_imgs_train"):
    for f in files:
        imgs.append(os.path.join(path, f))
for path, _, files in os.walk(root_path + "table_mask_train"):
    for f in files:
        masks.append(os.path.join(path, f))
train_generator = generator_disk(image_filenames, mask_filenames, batch_size=4, img_size=(input_height, input_width), n_calsses=n_classes, loss=loss)
'''

# from uploaded
image_filenames = list(map(lambda s: "img_table/" + s, sorted(os.listdir('img_table'))))
mask_filenames = list(map(lambda s: "img_mask_column/" + s, sorted(os.listdir('img_mask_column'))))
image_filenames_val = list(map(lambda s: "img_table_val/" + s, sorted(os.listdir('img_table_val'))))
mask_filenames_val = list(map(lambda s: "img_mask_column_val/" + s, sorted(os.listdir('img_mask_column_val'))))
train_generator = image_generator(image_filenames, mask_filenames, batch_size=batch_size, img_size=(input_height, input_width), n_calsses=n_classes, loss=loss)
val_generator = image_generator(image_filenames_val, mask_filenames_val, batch_size=batch_size, img_size=(input_height, input_width), n_calsses=n_classes, loss=loss)

t = train_generator.__getitem__(0)
t[0].shape, t[1].shape

((4, 1024, 1024, 3), (4, 1048576, 6))

In [0]:
# model

In [0]:
# test 1
'''
img_input = Input(shape=(input_height,input_width , 3 ))

conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(img_input)
conv1 = Dropout(0.2)(conv1)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D((2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Dropout(0.2)(conv2)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D((2, 2))(conv2)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
conv3 = Dropout(0.2)(conv3)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)

up1 = concatenate([UpSampling2D((2, 2))(conv3), conv2], axis=-1)
conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(up1)
conv4 = Dropout(0.2)(conv4)
conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)

up2 = concatenate([UpSampling2D((2, 2))(conv4), conv1], axis=-1)
conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(up2)
conv5 = Dropout(0.2)(conv5)
conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv5)
# conv5 = Conv2D(32, (3, 3), activation='softmax', padding='same')(conv5)


out = Conv2D(n_classes, (1, 1) , padding='same')(conv5)
'''

'''
# test 2
img_input = Input(shape=(input_height,input_width , 3 ))

conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(img_input)
conv1 = Dropout(0.2)(conv1)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D((4, 4))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Dropout(0.2)(conv2)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D((2, 2))(conv2)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
conv3 = Dropout(0.3)(conv3)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)

u3 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (conv3)
up1 = concatenate([u3, conv2], axis=-1)

conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(up1)
conv4 = Dropout(0.3)(conv4)
conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)
# pool3 = MaxPooling2D((2, 2))(conv4)
conv5 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)
conv5 = Dropout(0.2)(conv5)
conv5 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv5)

u4 = Conv2DTranspose(32, (2, 2), strides=(4, 4), padding='same') (conv5)
up2 = concatenate([u4, conv1], axis=-1)

conv6 = Conv2D(32, (3, 3), activation='relu', padding='same')(up2)
conv6 = Dropout(0.1)(conv6)
conv6 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv6)

out = Conv2D(n_classes, (1, 1) , padding='same')(conv6)

out = Reshape((input_height*input_width, -1))(out)
out = Activation('softmax')(out)
'''

#test 3
img_input = Input(shape=(input_height,input_width , 3 ))

conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(img_input)
conv1 = Dropout(0.5)(conv1)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D((4, 4))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Dropout(0.5)(conv2)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)

pool2 = MaxPooling2D((2, 2))(conv2)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Dropout(0.5)(conv2)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)

pool2 = MaxPooling2D((2, 2))(conv2)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
conv3 = Dropout(0.5)(conv3)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)

u3 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (conv3)
up1 = concatenate([u3, conv2], axis=-1)

conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(up1)
conv4 = Dropout(0.5)(conv4)
conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)
# pool3 = MaxPooling2D((2, 2))(conv4)
conv5 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)
conv5 = Dropout(0.5)(conv5)
conv5 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv5)

u4 = Conv2DTranspose(32, (2, 2), strides=(4, 4), padding='same') (conv5)
up2 = concatenate([u4, conv1], axis=-1)

conv6 = Conv2D(32, (3, 3), activation='relu', padding='same')(up2)
conv6 = Dropout(0.5)(conv6)
conv6 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv6)

out = Conv2D(n_classes, (1, 1) , padding='same')(conv6)

out = Reshape((input_height*input_width, -1))(out)
out = Activation('softmax')(out)

In [0]:
model = Model(img_input, out)
# model.compile(optimizer='adadelta', loss=loss,  metrics=['accuracy'])
model.compile(optimizer='adam', loss=loss, metrics=['accuracy', 'categorical_accuracy'])
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 1024, 1024, 3 0                                            
__________________________________________________________________________________________________
conv2d_61 (Conv2D)              (None, 1024, 1024, 3 896         input_5[0][0]                    
__________________________________________________________________________________________________
dropout_29 (Dropout)            (None, 1024, 1024, 3 0           conv2d_61[0][0]                  
__________________________________________________________________________________________________
conv2d_62 (Conv2D)              (None, 1024, 1024, 3 9248        dropout_29[0][0]                 
____________________________________________________________________________________________

In [0]:
# fit generator
filepath = root_path + "outputs/model_best_dominik_column_3_2_adam.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max', period=1)
# reduces learning rate on plateau
lr_reducer = ReduceLROnPlateau(factor=0.1,
                               cooldown=4,
                               monitor='val_loss',
                               patience=4, verbose=1,
                               min_lr=0.1e-5)
# stop learining as metric on validatopn stop increasing
early_stopping = EarlyStopping(patience=10, verbose=1, mode = 'auto')
callbacks_list = [checkpoint, lr_reducer]

In [0]:
history = model.fit_generator(train_generator, validation_data=val_generator, epochs=40, steps_per_epoch=512, callbacks=callbacks_list, workers=2, use_multiprocessing=True, verbose=1)

Epoch 1/40


Process ForkPoolWorker-230:
Process ForkPoolWorker-229:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-228:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-227:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call la

Epoch 1/40


  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py", line 406, in get_index
    return _SHARED_SEQUENCES[uid][i]
  File "<ipython-input-6-2927a09c0407>", line 36, in __getitem__
    np.array([self.get_segmentation_mask(file_name) for file_name in batch_y]))
  File "<ipython-input-6-2927a09c0407>", line 36, in <listcomp>
    np.array([self.get_segmentation_mask(file_name) for file_name in batch_y]))
  File "<ipython-input-6-2927a09c0407>", line 25, in get_segmentation_mask
    seg_labels[:, :, c] = (img == c).astype(int)
KeyboardInterrupt


Epoch 1/40


KeyboardInterrupt: ignored

In [0]:
def predict(model, file_name_img, file_name_mask, input_height, input_width,
            n_classes):
    img_test = cv2.imread(file_name_img, 1)
    img_mask = cv2.imread(file_name_mask, 0)
    img_mask = cv2.resize(img_mask, (input_height, input_width),
                          interpolation=cv2.INTER_NEAREST)
    img_test = cv2.resize(img_test, (input_height, input_width),
                          interpolation=cv2.INTER_NEAREST)
    img_test = np.expand_dims(img_test, axis=0)
    mask_pred = model.predict(img_test)
    mask_pred = mask_pred.reshape((input_height, input_width, n_classes)).\
        argmax(axis=2)
    plt.figure(figsize=(32, 32))
    plt.subplot(1, 3, 1)
    plt.imshow(img_test[0])
    plt.subplot(1, 3, 2)
    plt.imshow(img_mask)
    plt.subplot(1, 3, 3)
    plt.imshow(mask_pred)
    plt.show()


predict(model, "img_table_val/100_table.png", "img_mask_column_val/100_table.png", input_height, input_width, n_classes)